In [38]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients.",
        ),
        ("human", "I want to cook {cuisine} food"),
    ]
)
chef_chain = chef_prompt | chat

In [37]:
veg_chef_propmpt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a vegetarian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it.",
        ),
        ("human", "{recipe}"),
    ]
)

veg_chain = veg_chef_propmpt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({"cuisine": "Indian"})

AIMessageChunk(content="For a vegetarian version of Chicken Tikka Masala, you can replace the chicken with a plant-based alternative such as tofu or paneer. Here's how you can prepare these alternatives:\n\n1. **Tofu**: \n   - Use firm or extra-firm tofu for this recipe.\n   - Drain the tofu and press it to remove excess water. You can do this by wrapping the tofu block in a clean kitchen towel or paper towels and placing a heavy object on top for about 15-30 minutes.\n   - Cut the tofu into bite-sized cubes and follow the same marinating process as you would with chicken. Tofu absorbs flavors well, so the marination time can be similar to chicken.\n   - Instead of baking, you can pan-fry the marinated tofu in a bit of oil until it's golden brown and slightly crispy.\n\n2. **Paneer**:\n   - Paneer is a type of Indian cheese that holds its shape well when cooked.\n   - Cut the paneer into cubes and lightly pan-fry them until they develop a golden crust. This step helps the paneer hold i